In [1]:
import pandas as pd
import numpy as np
import datetime
import re

Считываем таблицу с результатами гонки (предварительно в экселе отрезал все лишнеее, оставив только результаты по гонке с raceId = 841)

In [2]:
df = pd.read_csv(os.path.join(__location__, ,'lap_times.csv'))
df.head(5)

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


Задание 1. Суммарное время заезда для каждого гонщика.

Заметил, что некоторые гонщики имеют менее 57 пройденных кругов гонки. Как я понял, это значит, что они не доехали до финиша. Решил учесть это при решении

In [3]:
LAPS = 58
df['status'] = df.lap.apply(lambda x: 'finished' if x>=LAPS else 'inRace')
#теперь гонщикам, у которых максимальное значение по
#пройденным кругам меньше 58, назначим статус leftRace на их последнем пройденном круге
leftRacers = df.groupby('driverId').lap.max() [df.groupby('driverId').lap.max() < LAPS]
for driver in leftRacers.index:
    df.loc[(df['driverId']==driver)&(df['lap']==leftRacers[driver]),'status'] = 'leftRace'
#суммарное время заезда для финишировавших гонщиков
print('Время закончивших гонку\n',\
      df.loc[-df['driverId'].isin(leftRacers.index),:]\
                            .groupby('driverId')\
                            .milliseconds.sum()\
                            .apply(lambda x: str(datetime.timedelta(milliseconds = x)))\
                            .sort_values()\
                            .to_frame()\
                            .rename({'milliseconds': 'time'}, axis='columns'))
#суммарное время заезда для недоехавших
print('Время сошедших с дистанции\n',\
      df.loc[df['driverId'].isin(leftRacers.index),:]\
                           .groupby('driverId')\
                           .milliseconds.sum()\
                           .apply(lambda x: str(datetime.timedelta(milliseconds = x)))\
                           .sort_values()\
                           .to_frame()\
                           .rename({'milliseconds': 'time'}, axis='columns'))

Время закончивших гонку
                     time
driverId                
20        1:29:30.259000
1         1:29:52.556000
808       1:30:00.819000
4         1:30:02.031000
17        1:30:08.430000
18        1:30:24.563000
815       1:30:36.104000
155       1:30:47.131000
13        1:30:55.445000
Время сошедших с дистанции
                     time
driverId                
813       0:14:41.936000
5         0:31:16.459000
30        0:32:13.487000
3         0:35:21.876000
22        1:17:23.256000
816       1:29:34.518000
67        1:29:38.705000
16        1:29:54.794000
814       1:30:23.766000
153       1:30:28.923000
2         1:30:43.901000
15        1:30:53.536000
10        1:31:02.637000


Задание 2. Топ-3 гонщика по каждому кругу.

In [4]:
for lap in df.lap.unique():
    print('Lap', lap, 'top-3\n',  df.loc[df['lap']==lap,df.columns]\
                                                           .nsmallest(3,'milliseconds')\
                                                           .loc[:,['driverId','time']]\
                                                           .set_index('driverId'),'\n')

Lap 1 top-3
               time
driverId          
20        1:38.109
1         1:40.573
17        1:41.467 

Lap 2 top-3
               time
driverId          
20        1:33.006
17        1:33.725
1         1:33.774 

Lap 3 top-3
               time
driverId          
20        1:32.713
1         1:32.900
17        1:33.208 

Lap 4 top-3
               time
driverId          
1         1:32.582
20        1:32.803
17        1:32.933 

Lap 5 top-3
               time
driverId          
20        1:32.342
1         1:32.471
17        1:32.899 

Lap 6 top-3
               time
driverId          
1         1:32.434
20        1:32.605
17        1:32.916 

Lap 7 top-3
               time
driverId          
1         1:32.447
20        1:32.502
17        1:32.900 

Lap 8 top-3
               time
driverId          
1         1:32.310
20        1:32.537
4         1:33.099 

Lap 9 top-3
               time
driverId          
1         1:32.612
20        1:33.240
17        1:33.330 

Lap 10 top

Задание 3. Для каждого гонщика вывести круг, в котором у него был самый большой
положительный рывок

In [5]:
#Для каждого гонщика посчитаем его рывки по каждому кругу(для первого круга рывок=-1)
#и позицию на предыдущем круге(для отображения в будущем)
df['spurt'] = (df.position.shift(1) - df.position).fillna(-1).astype(int)
df['prev_position'] = df.position.shift(1).fillna(-1).astype(int)
df.loc[df['lap']==1,['spurt','prev_position']] = [-1,-1]
#Составим Series для каждого гонщика с его максимальным рывком (положительным)
max_spurt = df.groupby('driverId').spurt.max()[df.groupby('driverId').spurt.max()>0]
print('Дучшие отыгрывания позиций')
for driver in max_spurt.index:
    print('Гонщик',driver,'\n',df.loc[df.index[(df['driverId']==driver)&\
         (df['spurt']==max_spurt[driver])],['driverId','lap','spurt','position','prev_position']]\
         .set_index('driverId'),'\n')

Дучшие отыгрывания позиций
Гонщик 1 
           lap  spurt  position  prev_position
driverId                                     
1          14      1         1              2 

Гонщик 2 
           lap  spurt  position  prev_position
driverId                                     
2          23      3        12             15 

Гонщик 3 
           lap  spurt  position  prev_position
driverId                                     
3          12      2         6              8 

Гонщик 4 
           lap  spurt  position  prev_position
driverId                                     
4          14      3        10             13
4          16      3         6              9 

Гонщик 5 
           lap  spurt  position  prev_position
driverId                                     
5          10      1        16             17
5          13      1        16             17
5          14      1        15             16
5          15      1        14             15
5          16      1        13      

Задание 4. Динамика мест по кругам для гонщиков, завершивших заезд в десятке лучших

In [7]:
#всего завершивших гонку гонщиков - 9 штук.
fin_result = df.loc[-df['driverId'].isin(leftRacers.index),:]\
                                   .groupby('driverId')\
                                   .milliseconds.sum()\
                                   .nlargest(10)\
                                   .sort_values(ascending = True)
for driver in fin_result.index:
    print('Гонщик ', driver,'\n', df.loc[df['driverId']==driver,['driverId','lap','position']]\
                                    .set_index('driverId'),'\n')

Гонщик  20 
           lap  position
driverId               
20          1         1
20          2         1
20          3         1
20          4         1
20          5         1
20          6         1
20          7         1
20          8         1
20          9         1
20         10         1
20         11         1
20         12         1
20         13         1
20         14         3
20         15         4
20         16         2
20         17         1
20         18         1
20         19         1
20         20         1
20         21         1
20         22         1
20         23         1
20         24         1
20         25         1
20         26         1
20         27         1
20         28         1
20         29         1
20         30         1
20         31         1
20         32         1
20         33         1
20         34         1
20         35         1
20         36         1
20         37         1
20         38         1
20         39         1
20 